In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class BrowserHandler:
    browser_instance = None

    @classmethod
    def initialize_browser(cls):
        cls.browser_instance = webdriver.Chrome()
        cls.browser_instance.maximize_window()
        cls.browser_instance.implicitly_wait(10)
        cls.browser_instance.get("https://app-staging.nokodr.com/")

    @classmethod
    def close_browser(cls):
        if cls.browser_instance:
            cls.browser_instance.quit()

    @classmethod
    def perform_signup(cls):
        wait = WebDriverWait(cls.browser_instance, 10)
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "(//input[contains(@class, 'slds-input')])[3]")))
        wait.until(EC.element_to_be_clickable(
            (By.CLASS_NAME, "slds-checkbox__label"))).click()
        wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@class='slds-col slds-size_1-of-1']"))).click()

class UserAuth(BrowserHandler):

    @classmethod
    def login_attempt(cls, user_id, user_key):
        cls.initialize_browser()
        email_input = user_id.strip()
        password_input = user_key.strip()

        wait = WebDriverWait(cls.browser_instance, 10)
        email_field = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//input[@name='username']")))
        password_field = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//input[@name='password']")))
        
        email_field.send_keys(email_input)
        password_field.send_keys(password_input)
        
        wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@title='Log In']"))).click()

        try:
            error_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//div[@class='content-margin']")))
            error_text = error_container.text
            if "Invalid Email or Password" in error_text:
                print(f"❌ Login failed for {email_input}: Invalid credentials.")
            elif "Please enter a valid email" in error_text:
                print(f"❌ Login failed for {email_input}: Incorrect email format.")
            else:
                print(f"❌ Login encountered an unexpected error for {email_input}: {error_text}")
        except Exception:
            print(f"✅ Login successful for {email_input}!")
        finally:
            cls.close_browser()

if __name__ == "__main__":
    # Reduced test cases
    login_attempts = [
        ("john.doe@gmail.com", "JohnDoe@123"),
        ("anujjagtap9696@gmail.com", "Anuj#9696"),
        ("user@example.com", "password123")
    ]

    for user_id, user_key in login_attempts:
        print(f"\n🔍 Testing login: Email: '{user_id}', Password: '{user_key}'")
        UserAuth.login_attempt(user_id, user_key)
